# Plot Barcelona streets

In [ ]:
from geopandas import GeoDataFrame
import pandas as pd
from shapely.geometry import LineString
import typing as T
from itertools import zip_longest
import os
import folium

In [ ]:
def convert_line_to_wkt(line: str) -> LineString:
    splitted_line = line.split(",")
    reshaped_line = reshape_line(splitted_line, chunksize=2, fillvalue=None)
    return LineString(reshaped_line)
    
def reshape_line(line: T.List[str], chunksize: int, fillvalue: int) -> T.Iterable[T.Tuple[float]]:
    """Reshape a line to match the LineString WKT format
    
    This is based on `zip_longest`, read more in 
    <https://docs.python.org/3/library/itertools.html#itertools.zip_longest> and
    in the StackOverflow solution posted in <https://stackoverflow.com/a/434411/5819113>
    
    
        zip_longest('ABCD', 'xy', fillvalue='-') --> Ax By C- D-
       
    """
    args = [iter(float(el) for el in line)] * chunksize
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
streets = pd.read_csv("../data/transit_relacio_trams.csv", sep=",")
# let's process the whole column
coords_wkt = streets["Coordenades"].apply(lambda line: convert_line_to_wkt(line))
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(streets, crs=crs, geometry=coords_wkt)
ax = gdf.plot(cmap="viridis")
barcelona_map = folium.Map([41.3947,2.1557], zoom_start=12.4, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(barcelona_map)

In [ ]:
list_schools = ['EEABB', 'EEBE', 'EETAC', 'EPSEB', 'EPSEM', 'ESEIAAT', 'ETSAB', 'ETSECCPB', 'ETSEIB', 'ETSETB', 'FIB', 'FME', 'FNB', 'FOOT']
data = pd.DataFrame({
   'lon':[1.9866716999999983, 
          2.2231519345468, 
          1.9872505, 
          2.1130787881117503, 
          1.8290409554431237, 
          2.020260374655512,
          2.115296240644385, 
          2.1114770846571855,
         2.11563799713493,
         2.112271912477879,
         2.113366253609662,
         2.1156621824795994,
         2.184497723790824,
         2.0236185376204054],
   'lat':[41.275919200000004, 
          41.414261499999995, 
          41.2755674, 
          41.3835842, 
          41.737127900000004, 
          41.56258005, 
          41.384277749999995, 
          41.38902913883299,
         41.38491311671433,
         41.38881605868467,
         41.38950017997591,
         41.38390531398507,
         41.38396190629874,
         41.56884538208665],
   'Name': list_schools
   #'Num_of_students':[10, 12, 40, 70, 23, 43, 100, 43]
}, dtype=str)
# add marker one by one on the map
for i in range(0,len(data)):
   folium.Marker(
      location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
      popup=data.iloc[i]['Name'],
   ).add_to(barcelona_map)

# Show the map again
barcelona_map

# Municipalities

In [ ]:
import geopandas as gpd

path = "../data/municipios.gdb"
spain = gpd.read_file(path)
municipalities = pd.read_csv("../data/codigos_postales_municipios.csv")
spain = pd.merge(spain, municipalities.astype({'municipio_id': 'string','codigo_postal': 'string' }), how='left', left_on=['CODIGOINE'], right_on=['municipio_id'])
spain.head()


In [ ]:
folium.GeoJson(spain).add_to(barcelona_map)
barcelona_map

# Renta (renta disponible)

In [ ]:
renta_data = pd.read_csv("../data/renta.csv")
renta_data['Unnamed: 0'] = renta_data['Unnamed: 0'].str[0:5] # WE loose names for zip code regions in this step
renta_data = renta_data.rename(columns={'Unnamed: 0': 'ZipCode'})
renta_data['ZipCode'] = pd.to_numeric(renta_data['ZipCode'], errors='coerce')
renta_data = renta_data[renta_data['ZipCode'].notna()].astype({'ZipCode': int})
renta_data